# https://blog.naver.com/rising_n_falling/221325080333

In [6]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [7]:
client_id = "j2nijytx8x"
client_pw = "D12z0TZrLISsTlZMNaDejzgrvYZnbn4rhdViIoKz"

In [25]:
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

data = pd.read_excel('cleanhouse.xlsx', usecols='B, C',names=['도움센터명칭','주소'])

# 위도경도 찾기
geo_coordi = []
for add in data['주소']:
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID',client_id)
    request.add_header('X-NCP-APIGW-API-KEY',client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
            else:
                print("'result' not exist!")
                latitude = None
                longitude  = None
        else:
            print('Response error code : %d' %rescode)
            latitude = None
            longitude = None
            
    geo_coordi.append([latitude, longitude])

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!


# https://blog.naver.com/rising_n_falling/221596375078

In [26]:
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"도움센터명칭":data["도움센터명칭"].values,
                              "도로명":data['주소'].values,
                              "위도": np_geo_coordi[:,0],
                              "경도": np_geo_coordi[:,1]})

In [27]:
writer = pd.ExcelWriter('clean_help.xlsx')
pd_geo_coordi.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [32]:
import folium

map = folium.Map(location=[33.365210, 126.529101],zoom_start=10)
for clean, la, ln in zip(data["도움센터명칭"], np_geo_coordi[:,0], np_geo_coordi[:,1]):
    m = folium.Marker([la,ln], popup=clean)
    m.add_to(map)
map